This notebook was prepared by [Donne Martin](https://github.com/donnemartin). Source and license info is on [GitHub](https://github.com/donnemartin/interactive-coding-challenges).

# Challenge Notebook

## Problem: Given a knapsack with a total weight capacity and a list of items with weight w(i) and value v(i), determine which items to select to maximize total value.

* [Constraints](#Constraints)
* [Test Cases](#Test-Cases)
* [Algorithm](#Algorithm)
* [Code](#Code)
* [Unit Test](#Unit-Test)
* [Solution Notebook](#Solution-Notebook)

## Constraints

* Can we replace the items once they are placed in the knapsack?
    * No, this is the 0/1 knapsack problem
* Can we split an item?
    * No
* Can we get an input item with weight of 0 or value of 0?
    * No
* Can we assume the inputs are valid?
    * No
* Are the inputs in sorted order by val/weight?
    * Yes, if not we'd need to sort them first
* Can we assume this fits memory?
    * Yes

## Test Cases

* items or total weight is None -> Exception
* items or total weight is 0 -> 0
* General case

<pre>
total_weight = 8
items
  v | w
  0 | 0
a 2 | 2
b 4 | 2
c 6 | 4
d 9 | 5

max value = 13
items
  v | w
b 4 | 2
d 9 | 5 
</pre>

## Algorithm

Refer to the [Solution Notebook]().  If you are stuck and need a hint, the solution notebook's algorithm discussion might be a good place to start.

    This is a classic MILP problem. Mixed integer formulation of this problem is:

    max (sum of value of all included items)
    s.t. total weight of all items must be < max_weight
    s.t. selection of items is binary (either picked or not picked, cannot have halves or fractions)
    
    To solve this problem with dynamic programming, we build a table, where we tabulate the effect
    of including an item
    
    


## Code

In [3]:
class Item(object):

    def __init__(self, label, value, weight):
        self.label = label
        self.value = value
        self.weight = weight

    def __repr__(self):
        return self.label + ' v:' + str(self.value) + ' w:' + str(self.weight)

In [7]:
class Knapsack(object):
    def fill_knapsack(self, input_items, total_weight):
        if input_items is None or total_weight is None:
            raise TypeError('Inputs cannot be none.')
        if not input_items or total_weight == 0:
            return 0
        items = list([Item(label='',value = 0, weight = 0)] + input_items)
        n_rows = len(input_items) + 1
        n_cols = total_weight + 1
        
        T = [[0] * n_cols for _ in range(n_rows)]
        for row in range(n_rows):
            for col in range(n_cols):
                if row == 0 or col == 0:
                    T[row][col] = 0 # the null item case and the null weight case is always 0
                # if we still got room to put the item
                elif col >= items[row].weight:
                    # we only pick the new item in the current row if it gives us a better value
                    T[row][col] = max(items[row].value + T[row-1][col-items[row].weight],
                                     T[row - 1][col])
                else:
                    T[row][col] = T[row-1][col] # otherwise keep the old row
        
        # now we have the weight table, we walk backwards to identify the items
        results = []
        i = n_rows - 1
        j = n_cols - 1
        while T[i][j] != 0:
            if T[i-1][j] == T[i][j]:
                i-=1
            elif T[i][j-1] == T[i][j]:
                j-=1
            else:
                results.append(items[i])
                i -= 1
                j -= items[i].weight
        return results        

# The algorithm for recursive implementation

The idea, given the problem you stated (which specifies we must use recursion) is simple: for each item that you can take, see if it's better to take it or not. So there are only two possible path:

you take the item
you don't take it
When you take the item, you remove it from your list and you decrease the capacity by the weight of the item.

When you don't take the item, you remove if from you list but you do not decrease the capacity.

Sometimes it helps to print what the recursive calls may look like. In this case, it could look like this:

    Calling 11 8 7 6 5  with cap: 20
     +Calling 8 7 6 5  with cap: 20
     |  Calling 7 6 5  with cap: 20
     |    Calling 6 5  with cap: 20
     |      Calling 5  with cap: 20
     |      Result: 5
     |      Calling 5  with cap: 14
     |      Result: 5
     |    Result: 11
     |    Calling 6 5  with cap: 13
     |      Calling 5  with cap: 13
     |      Result: 5
     |      Calling 5  with cap: 7
     |      Result: 5
     |    Result: 11
     |  Result: 18
     |  Calling 7 6 5  with cap: 12
     |    Calling 6 5  with cap: 12
     |      Calling 5  with cap: 12
     |      Result: 5
     |      Calling 5  with cap: 6
     |      Result: 5
     |    Result: 11
     |    Calling 6 5  with cap: 5
     |      Calling 5  with cap: 5
     |      Result: 5
     |    Result: 5
     |  Result: 12
     +Result: 20
      Calling 8 7 6 5  with cap: 9
        Calling 7 6 5  with cap: 9
          Calling 6 5  with cap: 9
            Calling 5  with cap: 9
            Result: 5
            Calling 5  with cap: 3
            Result: 0
          Result: 6
          Calling 6 5  with cap: 2
            Calling 5  with cap: 2
            Result: 0
          Result: 0
        Result: 7
        Calling 7 6 5  with cap: 1
          Calling 6 5  with cap: 1
            Calling 5  with cap: 1
            Result: 0
          Result: 0
        Result: 0
      Result: 8
    Result: 20


I did on purpose show the call to [8 7 6 5] with a capacity of 20, which gives a result of 20 (8 + 7 + 5).

Note that [8 7 6 5] is called twice: once with a capacity of 20 (because we didn't take 11) and once with a capacity of 9 (because with did take 11).

So the path to the solution:

11 not taken, calling [8 7 6 5] with a capacity of 20

8 taken, calling [7 6 5] with a capacity of 12 (20 - 8)

7 taken, calling [6 5] with a capacity of 5 (12 - 7)

6 not taken, calling [5] with a capacity of 5

5 taken, we're at zero.

The actual method in Java can fit in very few lines of code.

In [62]:
# recursive implementation
class KnapsackTopDown(object):
    def fill_knapsack(self, items, total_weight):
        if items is None or total_weight is None:
            raise TypeError('input_items or total_weight cannot be None')
        if not items or not total_weight:
            return 0
        memo = {}
        result = self._fill_knapsack(items, total_weight, memo, index=0)
        curr_item = result.item
        curr_weight = curr_item.weight
        picked_items = [curr_item]
        while curr_weight > 0:
            total_weight -= curr_item.weight
            curr_item = memo[(total_weight, len(items) - len(picked_items))].item
        return result
            
    def _fill_knapsack(self, items, total_weight, memo, index):
        if total_weight < 0:
            return Result(total_weight=0, item=None)
        if not total_weight or index >= len(items):
            return Result(total_weight=items[index - 1].value, item=items[index - 1])
        if (total_weight, len(items) - index - 1) in memo:
            weight=memo[(total_weight, 
                         len(items) - index - 1)].total_weight + items[index - 1].value
            return Result(total_weight=weight,
                          item=items[index-1])
        results = []
        for i in range(index, len(items)):
            total_weight -= items[i].weight
            result = self._fill_knapsack(items, total_weight, memo, index=i + 1)
            total_weight += items[i].weight
            results.append(result)
        results_index = 0
        for i in range(index, len(items)):
            memo[(total_weight, len(items) - i)] = max(results[results_index:])
            results_index += 1
        if index == 0:
            result_item = memo[(total_weight, len(items) - 1)].item
        else:
            result_item = items[index - 1]
        weight = max(results).total_weight + (items[index - 1].value if index > 0 else 0)
        return Result(total_weight=weight,
                      item=result_item)
        

In [63]:
t = KnapsackTopDown()
items = []
items.append(Item(label='a', value=2, weight=2))
items.append(Item(label='b', value=4, weight=2))
items.append(Item(label='c', value=6, weight=4))
items.append(Item(label='d', value=9, weight=5))
total_weight = 8
expected_value = 13
results = t.fill_knapsack(items, total_weight)
results

NameError: global name '_fill_knapsack' is not defined

## Unit Test

**The following unit test is expected to fail until you solve the challenge.**

In [8]:
# %load test_knapsack.py
from nose.tools import assert_equal, assert_raises


class TestKnapsack(object):

    def test_knapsack_bottom_up(self):
        knapsack = Knapsack()
        assert_raises(TypeError, knapsack.fill_knapsack, None, None)
        assert_equal(knapsack.fill_knapsack(0, 0), 0)
        items = []
        items.append(Item(label='a', value=2, weight=2))
        items.append(Item(label='b', value=4, weight=2))
        items.append(Item(label='c', value=6, weight=4))
        items.append(Item(label='d', value=9, weight=5))
        total_weight = 8
        expected_value = 13
        results = knapsack.fill_knapsack(items, total_weight)
        assert_equal(results[0].label, 'd')
        assert_equal(results[1].label, 'b')
        total_value = 0
        for item in results:
            total_value += item.value
        assert_equal(total_value, expected_value)
        print('Success: test_knapsack_bottom_up')

    def test_knapsack_top_down(self):
        knapsack = KnapsackTopDown()
        assert_raises(TypeError, knapsack.fill_knapsack, None, None)
        assert_equal(knapsack.fill_knapsack(0, 0), 0)
        items = []
        items.append(Item(label='a', value=2, weight=2))
        items.append(Item(label='b', value=4, weight=2))
        items.append(Item(label='c', value=6, weight=4))
        items.append(Item(label='d', value=9, weight=5))
        total_weight = 8
        expected_value = 13
        assert_equal(knapsack.fill_knapsack(items, total_weight), expected_value)
        print('Success: test_knapsack_top_down')

def main():
    test = TestKnapsack()
    test.test_knapsack_bottom_up()
    test.test_knapsack_top_down()


if __name__ == '__main__':
    main()

Success: test_knapsack_bottom_up


NameError: global name 'KnapsackTopDown' is not defined

## Solution Notebook

Review the [Solution Notebook]() for a discussion on algorithms and code solutions.